In [2]:
%matplotlib inline
import seaborn as sbn
sbn.set_style('white')
import numpy as np
from skimage.io import imsave
import matplotlib.pyplot as plt
from IPython.display import Image
from sklearn.manifold import TSNE

from keras.models import Model
from keras.layers import InputLayer
from keras import backend as K

from machinedesign.autoencoder.interface import load
from machinedesign.transformers import inverse_transform_one, transform_one
from machinedesign.viz import grid_of_images_default
from sklearn.ensemble import RandomForestClassifier
from imgs_to_video import imgs_to_video

Using Theano backend.


In [26]:
folder = 'celeba2_discrete'
model = load('../results/{}'.format(folder))

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 28, 28)        80064     
_________________________________________________________________
activation_1 (Activation)    (None, 64, 28, 28)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 24, 24)       204928    
_________________________________________________________________
activation_2 (Activation)    (None, 128, 24, 24)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 20, 20)       819456    
_________________________________________________________________
activation_3 (Activation)    (None, 256, 20, 20)       0         
__________

In [4]:
data = np.load('../../data/shoes.npz')
X = data['X']
X /= 255.

In [30]:
inp = model.get_layer('input_1').input
out = model.get_layer('activation_3').output
hid = model.get_layer('activation_3')
enc = K.function([inp], out)

inp = model.get_layer('activation_3').output
out = model.get_layer('axis_softmax_1').output

dec = K.function([inp], out)

In [9]:
h = np.zeros((64,)+hid.output_shape[1:])
h[np.arange(h.shape[0]), np.arange(h.shape[0]), 0, 0] = 2.0
y = dec([h])
y = inverse_transform_one(y, model.transformers)
im = grid_of_images_default(y)
imsave('../results/{}/features.png'.format(folder), im)


/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [11]:
x = transform_one(X[0:1], model.transformers)
h = enc([x])
nb = h.shape[1]
h = h * np.ones((nb + 1, 1, 1, 1))
for i in range(nb):
    h[i, i:] = 0.
#h[np.arange(len(h)), np.arange(len(h)), :, :] = 0.
print(h.shape)
y = dec([h])
y = inverse_transform_one(y, model.transformers)
im = grid_of_images_default(y)
imsave('../results/{}/features.png'.format(folder), im)
y = y.transpose((0, 2, 3, 1))
imgs_to_video(y, out='../results/{}/features.mp4'.format(folder))

(257, 256, 1, 1)


/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float32 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [147]:
x = transform_one(X[0:10], model.transformers)
h = enc([x])
y = dec([h])
y = inverse_transform_one(y, model.transformers)
im = grid_of_images_default(y)
imsave('results/{}/features.png'.format(folder), im)
y = y.transpose((0, 2, 3, 1))
imgs_to_video(y, out='results/{}/features.mp4'.format(folder))

/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float32 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [149]:
h = np.zeros((hid.output_shape[1],) + hid.output_shape[1:])
h[np.arange(len(h)), np.arange(len(h)), :, :] = 5.0
y = dec([h])
y = inverse_transform_one(y, model.transformers)
im = grid_of_images_default(y)
imsave('results/{}/features.png'.format(folder), im)
y = y.transpose((0, 2, 3, 1))
imgs_to_video(y, out='results/{}/features.mp4'.format(folder))

/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float32 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/io/_io.py:132: UserWarning: /tmp/img_to_videomxrc4syq/img00000148.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/io/_io.py:132: UserWarning: /tmp/img_to_videomxrc4syq/img00000149.png is a low contrast image
  warn('%s is a low contrast image' % fname)


In [ ]:
h = np.random.normal(size=(36,) + hid.output_shape[1:])
for _ in range(10):
    y = dec([h])
    y= inverse_transform_one(y, model.transformers)
    ypix = y
    y = transform_one(y, model.transformers)
    h = enc([y])
im = grid_of_images_default(ypix)
imsave('out.png'.format(folder), im)